In [3]:
import numpy as np 
import pandas as pd 

import os
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization,Activation,Input,UpSampling2D,Lambda
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import regularizers
import cv2
from tqdm import tqdm
import random
import numpy as np
from tensorflow.keras.utils import Sequence
import random
from keras import backend as K
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import resnet

In [25]:
BATCH_SIZE=8
IMG_SIZE=299

In [5]:
import zipfile
with zipfile.ZipFile('../input/notebook36cfb71da3/database-full.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')
os.remove('./data/database-full/ image11396.jpg')
import shutil
shutil.rmtree('./data/__MACOSX')

In [26]:
def load_image_from_path(path):
    
    image = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image)
    image = tf.image.resize(image, size = (IMG_SIZE, IMG_SIZE))
    
    return image/255.

In [27]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [28]:
def get_mapk(K,similarities,retrieval):
    scores = []
    for predicted, actual in zip(similarities, retrievals):
        scores.append(mapk(actual, predicted, k=K))
    return scores

In [29]:
def sorting(tup, reverse=True):
    return(sorted(tup, key = lambda x: x[1],reverse=reverse))
def get_most_similars(query, database):
    
    x2 = database[database.img_name == query].encoded.values[0]
    distances = []
    names = []
    for i, fila in database.iterrows():

        x1 = fila.encoded
        names.append(fila.img_name)
        distances.append(np.linalg.norm(x2-x1))
        
    distances = [(x,y) for x,y in zip(names, distances)]
    distances=sorting(distances,False)
        
    return np.array(distances)[:,0]

In [10]:
example = pd.read_csv('../input/taller3data/examples_of_retrieval.csv', sep="\t")
retrievals=[]
retrievals.append([x.replace(" ", "") for x in example.columns[0].split(",")])
for i in example.index:
    retrievals.append([x.replace(" ", "") for x in example.iloc[i].values[0].split(",")])

In [30]:
img_gen=ImageDataGenerator(rescale=1./255.)
img_data=img_gen.flow_from_directory('./data',shuffle=False,target_size=(299,299),class_mode='input',batch_size=BATCH_SIZE)
img_name=img_names=[x.split(' ')[1] for x in img_data.filenames]

In [32]:
IncRes=tf.keras.applications.InceptionResNetV2(weights="imagenet", include_top=True, input_shape=(299, 299, 3))
model_1=Model(inputs=IncRes.input, outputs=IncRes.get_layer("avg_pool").output)

In [33]:
encoded_1=model_1.predict(img_data,verbose=1)

In [35]:
encoded_df_1=pd.DataFrame({'encoded':[x for x in encoded_1],'img_name':img_names})

In [39]:
encoded_df_1.encoded.values[0]

In [42]:
similarities_1 = []
for retrieval in tqdm(retrievals):
    query = retrieval[0]
    similarities_1.append(np.array(get_most_similars(query, encoded_df_1)[:100]))

In [45]:
most_sim_1=get_most_similars('image5876.jpg',encoded_df_1)

In [46]:
for im in most_sim_1[:10]:
    plt.imshow(load_image_from_path('data/database-full/ '+im))
    plt.title(im)
    plt.show()

In [47]:
Xception=tf.keras.applications.Xception(weights="imagenet", include_top=True, input_shape=(299, 299, 3))
model_2=Model(inputs=Xception.input, outputs=Xception.get_layer("avg_pool").output)

In [48]:
encoded_2=model_2.predict(img_data,verbose=1)

In [49]:
encoded_df_2=pd.DataFrame({'encoded':[x for x in encoded_2],'img_name':img_names})

In [50]:
similarities_2 = []
for retrieval in tqdm(retrievals):
    query = retrieval[0]
    similarities_2.append(np.array(get_most_similars(query, encoded_df_2)[:100]))

In [51]:
most_sim_2=get_most_similars('image5876.jpg',encoded_df_2)

In [52]:
for im in most_sim_2[:10]:
    plt.imshow(load_image_from_path('data/database-full/ '+im))
    plt.title(im)
    plt.show()

In [56]:
np.mean(get_mapk(100,similarities_1,retrievals))

In [55]:
np.mean(get_mapk(100,similarities_2,retrievals))

In [110]:
model_2.summary()

In [115]:
# for layer in model_2.layers[:-1]:
#     layer.trainable = False
# input_img = layers.Input(shape=(299,299,3))
# encoded = layers.
# encoded = layers.Dense(128, activation='relu')(input_img)
# encoded = layers.Dense(64, activation='relu')(encoded)
# encoded = layers.Dense(32, activation='relu')(encoded)

# decoded = layers.Dense(64, activation='relu')(encoded)
# decoded = layers.Dense(128, activation='relu')(decoded)
# decoded = layers.Dense(2048, activation='sigmoid')(decoded)

In [116]:
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# autoencoder.fit(x_train, x_train,
#                 epochs=100,
#                 batch_size=256,
#                 shuffle=True,
#                 validation_data=(x_test, x_test))

In [57]:
queries_test = pd.read_csv('../input/taller3data/names_test_queries.csv', sep="\t", header=None)

In [66]:
queries_test = pd.read_csv('../input/taller3data/names_test_queries.csv', sep="\t", header=None)
subm_similarities = []
for i, query_ in tqdm(queries_test.iterrows()):
    query = query_[0][1:]
    subm_similarities.append(np.array(get_most_similars(query, encoded_df_2)[1:101]))

In [90]:
final_sim=[]
for sim in subm_similarities:
    aux_l=[]
    for im in sim:
        aux_l.append(im[5:-4])
    final_sim.append(np.array(aux_l))
        

In [101]:
final = pd.DataFrame({'Id': [x[0][6:-4] for x in queries_test.values], 'Expected': [" ".join(x) for x in final_sim]})

In [105]:
final=final.set_index('Id')

In [107]:
final.to_csv('subm_xcept.csv')